<a href="https://colab.research.google.com/github/paruliansaragi/Abstractive-Text-Summarization/blob/master/ABS_1504_Fail_and_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!unzip './bbc-news-summary.zip'
#!unrar x 'BBC News Summary.rar'

##Attention

We use an encoder (usually a bi-dir GRU/LSTM) that encodes a sentence (word-embeddings of a sequence of tokens) and produces a series of hidden states that captures the meaning of a word in a given (potentially, global) context. I prefer to use pre-trained word embeddings rather than start from scratch. The next hidden state is calcualted from the previous state plus the new input embedding. The forward GRU will only know previous states and this is flipped for the backward GRU and by concatenating these two hidden states we are able to create a global contextual representation of a word in a given context. The decoder is the same as the encoder but also takes in a context vector. The attention mechanism selects the salient parts of the source sentence (from encoder) to predict the next word. This is done by comparing the last decoder state with each source hidden state, then this decoder state is put through a softmax to give us a prediction distribution. The loss is usually the cross-entropy loss which compares the correct distribution against the predicted one.

Since sequences will have variable lengths we need to use these PyTorch helper functions called `pack_padded_sequence` and `pad_packed_sequence`.
These functions take care of masking and padding, so that the resulting word representations are simply zeros after a sentence stops.

The code below reads in a source sentence (a sequence of word embeddings) and produces the hidden states.
It also returns a final vector, a summary of the complete sentence, by concatenating the first and the last hidden states (they have both seen the whole sentence, each in a different direction). We will use the final vector to initialize the decoder.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
rnn = torch.nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

The view function is meant to reshape the tensor. Say you have a tensor

In [0]:
a = torch.range(1, 16)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


a is a tensor that has 16 elements from 1 to 16(included). If you want to reshape this tensor to make it a 4 x 4 tensor then you can use

In [0]:
a = a.view(1, 1, -1)
a.shape, a#creates a long vector

(torch.Size([1, 1, 16]),
 tensor([[[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
           15., 16.]]]))

Now a will be a 4 x 4 tensor. Note that after the reshape the total number of elements need to remain the same. Reshaping the tensor a to a 3 x 5 tensor would not be appropriate. 

What is the meaning of parameter -1?

If there is any situation that you don't know how many rows you want but are sure of the number of columns, then you can specify this with a -1. (Note that you can extend this to tensors with more dimensions. Only one of the axis value can be -1). This is a way of telling the library: "give me a tensor that has these many columns and you compute the appropriate number of rows that is necessary to make this happen".

This can be seen in the neural network code that you have given above. After the line x = self.pool(F.relu(self.conv2(x))) in the forward function, you will have a 16 depth feature map. You have to flatten this to give it to the fully connected layer. So you tell pytorch to reshape the tensor you obtained to have specific number of columns and tell it to decide the number of rows by itself.

In [0]:
g = (training_pairs[0]).view(1, 1, -1)
g.shape[2], len(g)

(456, 1)

In [0]:
len(training_pairs[0])

456

In [0]:
embedding = nn.Embedding(voc.n_words, hidden_size)
#so this reshapes an embedding to 1 by 1 by as many rows
embedding

NameError: ignored

In [0]:
enc = EncoderRNN(voc.n_words, hidden_size)

In [0]:
emb = a.view(1, 1, -1)
emb.shape

torch.Size([1, 1, 456])

In [0]:
encoder1 = EncoderRNN(voc.n_words, hidden_size).to(DEVICE)

 Args:
        input_size: The number of expected features in the input `x`
        hidden_size: The number of features in the hidden state `h`
        num_layers: Number of recurrent layers. E.g., setting ``num_layers=2``
            would mean stacking two RNNs together to form a `stacked RNN`,
            with the second RNN taking in outputs of the first RNN and
            computing the final results. Default: 1
        nonlinearity: The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
        bias: If ``False``, then the layer does not use bias weights `b_ih` and `b_hh`.
            Default: ``True``
        batch_first: If ``True``, then the input and output tensors are provided
            as `(batch, seq, feature)`. Default: ``False``
        dropout: If non-zero, introduces a `Dropout` layer on the outputs of each
            RNN layer except the last layer, with dropout probability equal to
            :attr:`dropout`. Default: 0
        bidirectional: If ``True``, becomes a bidirectional RNN. Default: ``False``

    Inputs: input, h_0
        - **input** of shape `(seq_len, batch, input_size)`: tensor containing the features
          of the input sequence. The input can also be a packed variable length
          sequence. See :func:`torch.nn.utils.rnn.pack_padded_sequence`
          or :func:`torch.nn.utils.rnn.pack_sequence`
          for details.
        - **h_0** of shape `(num_layers * num_directions, batch, hidden_size)`: tensor
          containing the initial hidden state for each element in the batch.
          Defaults to zero if not provided. If the RNN is bidirectional,
          num_directions should be 2, else it should be 1.

In [0]:
encoder_output, encoder_hidden = encoder1(emb, enc_in)

In [0]:
emb.size(-1), enc_in.size(-1)

(456, 256)

In [0]:
rnn = torch.nn.GRU(456, 1, 256)

In [0]:
nn.Embedding()

torch.Size([1, 1, 456])

In [0]:
enc_in.shape

torch.Size([1, 1, 256])

In [0]:
output, hn = rnn(emb, enc_in)

In [0]:
rnn = torch.nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

In [0]:
encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)

In [0]:
#input of shape (seq_len, batch, input_size): 
#tensor containing the features of the input sequence.
rnn = torch.nn.GRU(1, 1, 456)

In [0]:
out, hid = rnn()

In [0]:
a.shape[0]

456

In [0]:
enc_in = encoder1.initHidden()

enc_hid, encout = encoder1(256, enc_in)

In [0]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ThisEncoderRNN(nn.Module):
  
  def __init__(self, embedded, embedding_dim, vocab_size, hidden_size):
    super(ThisEncoderRNN, self).__init__()
    self.hidden_size = hidden_size #Weight matrix of hidden is [hidden_size*hidden_size]
    
    self.embedding = nn.Embedding(embedding_dim, vocab_size)#Embedding_size, hidden_size
    self.gru = nn.GRU(vocab_size, hidden_size)
     #torch.nn.GRU(voc.n_words, hidden_size)
    
  def forward(self, input, hidden):
    #embedded = self.embedding(input).view(1, 1, -1)
    output = embedded
    output, hidden  = self.gru(output, hidden)#Feed GRU embedding, initial hidden state
    return output, hidden 
  
  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size)#, device=DEVICE)

In [0]:
enc1 = ThisEncoderRNN(embedded, embedding_dim, voc.n_words, 256)

In [0]:
# inut: shape (seq_len, batch, input_size): tensor
# h_0: shape (num_layers * num_directions, batch, hidden_size): tensor 
out, hid = enc1(s, in_hid)

In [0]:
#embedded: (src seq len, batch size, embed size)
#num_embeddings (int): size of the dictionary of embeddings
#embedding_dim (int): the size of each embedding vector
#padding_idx (int, optional): If given, pads the output with the embedding 
#vector at :attr:`padding_idx`
embedding_dim = 50
vocab_size = voc.n_words
embs = nn.Embedding(embedding_dim, vocab_size)

In [0]:
s = embs(src)

In [0]:
import numpy as np

lengths = len(s)

In [0]:
T, B, C = 100, 10, 4  # longest sequence, batch_size, feature_size
input = torch.randn(T, B, C, device='cuda')
lengths, _ = torch.sort(torch.randint(1, 256, (1,)), descending=True)

In [0]:
s = pack_padded_sequence(s, lengths)

In [0]:
s

In [0]:
# we will use CUDA if it is available
USE_CUDA = torch.cuda.is_available()
DEVICE=torch.device('cuda:0') # or set to 'cpu'
print("CUDA:", USE_CUDA)
print(DEVICE)

CUDA: True
cuda:0


In [0]:
s = src.view(1, 1, -1)

In [0]:
embedded = embs(s)

In [0]:
embedded.shape

torch.Size([1, 1, 6, 256])

In [0]:
in_hid = torch.zeros(1, 1, 256)

In [0]:
src.shape, embedded.shape, in_hid.shape, s.shape

(torch.Size([6, 1]),
 torch.Size([1, 1, 6, 256]),
 torch.Size([1, 1, 256]),
 torch.Size([1, 1, 6]))

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

packed = pack_padded_sequence(src, [6], batch_first=True)

In [0]:
rnn = torch.nn.GRU(voc.n_words, hidden_size)

In [0]:
s.shape, in_hid.shape

(torch.Size([6, 1, 8]), torch.Size([1, 1, 256]))

In [0]:
out, hid = rnn(s, in_hid)

In [0]:
s.dtype, in_hid.dtype

(torch.float32, torch.float32)

In [0]:
out, hid

In [0]:
some_src = ['This is some source text']
some_tgt = ['This is some target text']

In [0]:
src = " ".join(some_src)
tgt = " ".join(some_tgt)

In [0]:
voc = Vocab()
voc.addSentence(src)
voc.addSentence(tgt)

In [0]:
#voc.addSentence(some_src), voc.addSentence(some_tgt)
src, tgt = tensorFromSentence(voc, src), tensorFromSentence(voc, tgt)

In [0]:
enc1 = EncoderRNN(a.size(0), hidden_size).to(DEVICE)

In [0]:
enc_out, enc_hid = enc1.forward(a, enc.initHidden)

In [0]:
class DecoderRNN(nn.Module):
  
  def __init__(self, hidden_size, output_size):
    super(DecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)
    
  def forward(self, input, hidden):
    output = self.embedding(input).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden
  
  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [0]:
MAX_LENGTH = 10

class AttnDecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size, dropout=0.1, max_length=MAX_LENGTH):
    super(AttnDecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout = dropout
    self.max_length = max_length
    
    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.attn = nn.Linear(self.hidden_size *2, self.max_length)#*2 since its bidir
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout)
    self.gru = nn.GRU(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)
    
  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)
    
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
    # Unsqueeze:Returns a new tensor with a dimension of size one 
    # inserted at the specified position.
    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                            encoder_outputs.unsqueeze(0))
    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = F.log_softmax(self.out(output[0]), dim=1)
    return output, hidden, attn_weights
  
  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=DEVICE)
    

In [0]:
with open('./BBC News Summary/News Articles/politics/001.txt') as f:
  article = f.readlines()
with open('./BBC News Summary/Summaries/politics/001.txt') as f:
  summary = f.readlines()

In [0]:
article = " ".join(article)
article = article.replace('\n', '')
summary = " ".join(summary)

In [0]:
SOS_token = 0
EOS_token = 1

class Vocab:
  def __init__(self):
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: "SOS", 1: "EOS"}
    self.n_words = 2#Count the sos and eos tokens
  
  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)
    
  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

def indexesFromSentence(Vocab, sentence):
    return [Vocab.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(Vocab, sentence):
    indexes = indexesFromSentence(Vocab, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
voc = Vocab()
voc.addSentence(article)
voc.addSentence(summary)

In [0]:
a, s = tensorFromSentence(voc, article), tensorFromSentence(voc, summary)

In [0]:
len(s.unique()), len(a.unique()), voc.n_words

(120, 235, 243)

In [0]:
enc = EncoderRNN(voc.n_words, hidden_size).to(DEVICE)

In [0]:
g.size(-1)

456

In [0]:
f = nn.Embedding(input_size, hidden_size)

torch.Size([456, 1])

In [0]:
teacher_forcing = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim,
         criterion, max_length=MAX_LENGTH):
  encoder_hidden = encoder.initHidden()
  encoder_optim.zero_grad()
  decoder_optim.zero_grad()
  
  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)
  
  encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)
  
  loss = 0
  #accumulate encoder outputs 
  for i in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
    encoder_outputs[i] = encoder_output[0, 0]
    
  decoder_input = torch.tensor([[SOS_token]], device=DEVICE)
  
  decoder_hidden = encoder_hidden
  
  use_teacher_forcing = True if random.random() < teacher_forcing else False
  
  if use_teacher_forcing:
    #Feed target embedding as next input
    for i in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input,
                                                                 decoder_hidden,
                                                                 encoder_outputs)
      loss += criterion(decoder_output, target_tensor[i])
      decoder_input = target_tensor[i]
  else:
    #Use predictions as input
    for i in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input,
                                                                 decoder_hidden,
                                                                 encoder_outputs)
      #like np.argmax, index of highest probability
      topv, topi = decoder_output.topk(1)
      decoder_input = topi.squeeze().detach() #detach from history as input
      if decoder_input.item() == EOS_token:
        break
  
  loss.backward()
  
  encoder_optim.step()
  decoder_optim.step()
  
  return loss.item() / target_length#average the losses over all time steps/ preds

In [0]:
def prepareData(src, tgt, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [0]:
input_tensor = tensorFromSentence(voc, article)
input_tensor

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, lr=0.01):
  plot_losses = []
  print_loss_total = 0
  plot_loss_total = 0
  
  encoder_optimizer = optim.SGD(encoder.parameters(), lr=lr)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=lr)
  training_pairs = [tensorFromSentence(voc, article), tensorFromSentence(voc, summary)]
  criterion = nn.NLLLoss()
  
  for iter in range(1):
    #training_pair = training_pairs#[iter-1]
    input_tensor = tensorFromSentence(voc, article)
    target_tensor = tensorFromSentence(voc, summary)
    
    loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
                decoder_optimizer, criterion)
    
    print_loss_total += loss
    plot_loss_total += loss
    if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

       

Evaluation is the same as training but without targets so we simply feed the decoders preds back to itself for each time step. 

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)
    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.initHidden()
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=DEVICE)
    
    for i in range(input_length):
      encoder_output, encoder_hidden = encoder(input_tensor[i],
                                              encoder_hidden)
      encoder_outputs[i] += encoder_output[0, 10]
    
    decoder_input = torch.tensor([[SOS_token]], device=DEVICE)
    decoder_hidden = encoder_hidden
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)
    
    for di in range(max_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input,
                                                                 decoder_hidden,
                                                                 encoder_outputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoder_output.data.topk(1)
      if topi.item() == EOS_token:
        break
      else:
        decoded_words.append(voc.index2word[topi.item()])
        
      decoder_input = topi.squeeze().detach()
    return decoded_words, decoder_attentions[:di + 1]

In [0]:
from torch import optim
import random


hidden_size = 256
encoder1 = EncoderRNN(voc.n_words, hidden_size).to(DEVICE)
attn_decoder1 = AttnDecoderRNN(hidden_size, voc.n_words, dropout=0.1).to(DEVICE)

trainIters(encoder1, attn_decoder1, 5, 1)

IndexError: ignored

In [0]:
#@title Encoder { form-width: "25px" }
class Encoder(nn.Module):
    """Encodes a sequence of word embeddings"""
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, 
                          batch_first=True, bidirectional=True, dropout=dropout)
        
    def forward(self, x, mask, lengths):
        """
        Applies a bidirectional GRU to sequence of embeddings x.
        The input mini-batch x needs to be sorted by length.
        x should have dimensions [batch, time, dim].
        """
        #https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.pack_padded_sequence
        #takes an input (Tensor) – padded batch of variable length sequences.
        #lengths (Tensor) – list of sequences lengths of each batch element.
        #batch_first (bool, optional) – if True, the input is expected in B x T x * format.
        packed = pack_padded_sequence(x, lengths, batch_first=True)
        output, final = self.rnn(packed)#output from GRU is: output-tensor containing output features from last layer of GRU for each timestep,
        #and a tensor containing the hidden state for t = seq_len
        #https://pytorch.org/docs/stable/nn.html#pad-packed-sequence
        #Pads a packed batch of variable length sequences.
        #It is an inverse operation to pack_padded_sequence().
        #takes as input: sequence (PackedSequence) – batch to pad
        #batch_first (bool, optional) – if True, the output will be in B x T x * format.
        #returns: a Tuple of Tensor containing the padded sequence, 
        #and a Tensor containing the list of lengths of each sequence in the batch.
        output, _ = pad_packed_sequence(output, batch_first=True)

        # we need to manually concatenate the final states for both directions
        fwd_final = final[0:final.size(0):2]
        bwd_final = final[1:final.size(0):2]
        final = torch.cat([fwd_final, bwd_final], dim=2)  # [num_layers, batch, 2*dim]

        return output, final

The decoder is a conditional GRU which means that rather than starting from an empty state, its initial hidden state results from a projection of the encoder final vector. We use teacher forcing during trainng which is the trg_embed. We feed the correct previous target word embedding straight to the GRU at each time step. The `forward` returns all decoder hidden states and pre-output vectors used to compure the loss. After predicting a word from the pre-output vector we call it again but supplying the word embedding of the previously predicted word and the last state.  

You can use pretrained embeddings or learn them or fine tune them...

In [0]:
#@title Show Attention Map
def show_attention_map(src_words, pred_words, attention, pointer_ratio=None):
  fig, ax = plt.subplots(figsize=(16, 4))
  im = plt.pcolormesh(np.flipud(attention), cmap="GnBu")
  # set ticks and labels
  ax.set_xticks(np.arange(len(src_words)) + 0.5)
  ax.set_xticklabels(src_words, fontsize=14)
  ax.set_yticks(np.arange(len(pred_words)) + 0.5)
  ax.set_yticklabels(reversed(pred_words), fontsize=14)
  if pointer_ratio is not None:
    ax1 = ax.twinx()
    ax1.set_yticks(np.concatenate([np.arange(0.5, len(pred_words)), [len(pred_words)]]))
    ax1.set_yticklabels('%.3f' % v for v in np.flipud(pointer_ratio))
    ax1.set_ylabel('Copy probability', rotation=-90, va="bottom")
  # let the horizontal axes labelling appear on top
  ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
  # rotate the tick labels and set their alignment
  plt.setp(ax.get_xticklabels(), rotation=-45, ha="right", rotation_mode="anchor")


In [0]:
#@title Decoder { form-width: "25px" }
class Decoder(nn.Module):
    """A conditional RNN decoder with attention."""
    
    def __init__(self, emb_size, hidden_size, attention, num_layers=1, dropout=0.5,
                 bridge=True):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.attention = attention
        self.dropout = dropout
                 
        self.rnn = nn.GRU(emb_size + 2*hidden_size, hidden_size, num_layers,
                          batch_first=True, dropout=dropout)
                 
        # to initialize from the final encoder state
        self.bridge = nn.Linear(2*hidden_size, hidden_size, bias=True) if bridge else None

        self.dropout_layer = nn.Dropout(p=dropout)
        self.pre_output_layer = nn.Linear(hidden_size + 2*hidden_size + emb_size,
                                          hidden_size, bias=False)
        
    def forward_step(self, prev_embed, encoder_hidden, src_mask, proj_key, hidden):
        """Perform a single decoder step (1 word)"""

        # compute context vector using attention mechanism
        #somelist[-1] get the last element of a list
        #nn.Unsqueeze is useful and lets us insert the dimension without 
        #explicitly being aware of the other dimensions when writing the code.
        query = hidden[-1].unsqueeze(1)  # [#layers, B, D] -> [B, 1, D]
        context, attn_probs = self.attention(
            query=query, proj_key=proj_key,
            value=encoder_hidden, mask=src_mask)

        # update rnn hidden state
        rnn_input = torch.cat([prev_embed, context], dim=2)
        output, hidden = self.rnn(rnn_input, hidden)
        #Concatenates the given sequence of seq tensors in the given dimension.
        pre_output = torch.cat([prev_embed, output, context], dim=2)
        pre_output = self.dropout_layer(pre_output)
        #
        pre_output = self.pre_output_layer(pre_output)

        return output, hidden, pre_output
    
    def forward(self, trg_embed, encoder_hidden, encoder_final, 
                src_mask, trg_mask, hidden=None, max_len=None):
        """Unroll the decoder one step at a time."""
                                         
        # the maximum number of steps to unroll the RNN
        if max_len is None:
            max_len = trg_mask.size(-1)

        # initialize decoder hidden state with final encoder hidden states
        if hidden is None:
            hidden = self.init_hidden(encoder_final)
        
        # pre-compute projected encoder hidden states
        # (the "keys" for the attention mechanism)
        # this is only done for efficiency
        proj_key = self.attention.key_layer(encoder_hidden)
        
        # here we store all intermediate hidden states and pre-output vectors
        decoder_states = []
        pre_output_vectors = []
        
        # unroll the decoder RNN for max_len steps
        for i in range(max_len):
            prev_embed = trg_embed[:, i].unsqueeze(1)
            output, hidden, pre_output = self.forward_step(
              prev_embed, encoder_hidden, src_mask, proj_key, hidden)
            decoder_states.append(output)
            pre_output_vectors.append(pre_output)

        decoder_states = torch.cat(decoder_states, dim=1)
        pre_output_vectors = torch.cat(pre_output_vectors, dim=1)
        return decoder_states, hidden, pre_output_vectors  # [B, N, D]

    def init_hidden(self, encoder_final):
        """Returns the initial decoder state,
        conditioned on the final encoder state."""

        if encoder_final is None:
            return None  # start with zeros

        return torch.tanh(self.bridge(encoder_final))            


At each time step, the decoder has access to all source word representations. The attention mechanism allows the model to focus on the currently most relevant part of the source sentence. The state of the decoder is given by the GRU hidden state si. So if we want to know which source words hj are most relevant we need something with two inputs. We use a multilayer perceptron with tanh to both the current decoder state si (the query) and each encoder state hj(the key), and then project it into a single value (scalar) to get the attention energy eij. Once all energies are computed, we normalize them with softmax. Then the context vector for time step i is the weighted sum of the encoder hidden states (the values).

$$\mathbf{c}_i = \sum_j \alpha_{ij} \mathbf{h}_j$$

So you've essentially put a weight on each encoder hidden state (hj, the words) that we want to pay attention to. 

In [0]:
#@title
class BahdanauAttention(nn.Module):
    """Implements Bahdanau (MLP) attention"""
    
    def __init__(self, hidden_size, key_size=None, query_size=None):
        super(BahdanauAttention, self).__init__()
        
        # We assume a bi-directional encoder so key_size is 2*hidden_size
        key_size = 2 * hidden_size if key_size is None else key_size
        query_size = hidden_size if query_size is None else query_size

        self.key_layer = nn.Linear(key_size, hidden_size, bias=False)
        self.query_layer = nn.Linear(query_size, hidden_size, bias=False)
        self.energy_layer = nn.Linear(hidden_size, 1, bias=False)
        
        # to store attention scores
        self.alphas = None
        
    def forward(self, query=None, proj_key=None, value=None, mask=None):
        assert mask is not None, "mask is required"

        # We first project the query (the decoder state).
        # the query can be seen as asking what do we want to pay attention to?
        # The projected keys (the encoder states) were already pre-computated.
        query = self.query_layer(query)
        
        # Calculate energy scores.
        scores = self.energy_layer(torch.tanh(query + proj_key))
        scores = scores.squeeze(2).unsqueeze(1)
        
        # Mask out invalid positions.
        # The mask marks valid positions so we invert it using `mask & 0`.
        scores.data.masked_fill_(mask == 0, -float('inf'))
        
        # Turn scores to probabilities.
        alphas = F.softmax(scores, dim=-1)
        self.alphas = alphas        
        
        # The context vector is the weighted sum of the values.
        #a batch matrix-matrix product of matrices stored in batch1 and batch2.
        context = torch.bmm(alphas, value)
        
        # context shape: [B, 1, 2D], alphas shape: [B, 1, M]
        return context, alphas

In [0]:
embedded.shape

NameError: ignored

In [0]:
class MyEncoderRNN(nn.Module):
  
  def __init__(self, embed_size, hidden_size, dropout: float=0.5):
    super(MyEncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embed_size = embed_size
    self.rnn = nn.GRU(embed_size, hidden_size, bidirectional=2, dropout=dropout)
    
  def forward(self, embedding, hidden, inp_length=None):
    """
    Applies bidir GRU to sequqnece of embeddings.
    :param embedding: needs to be [src seq length x batchsize x embedding size]
    :param hidden: needs to be [number of directions(2) x batch size x encoder hidden size]
    """
    embedding = pack_padded_sequence(embedding, torch.LongTensor(1))
    #https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.pack_padded_sequence
    #takes an input (Tensor) – padded batch of variable length sequences.
    #lengths (Tensor) – list of sequences lengths of each batch element.
    #batch_first (bool, optional) – if True, the input is expected in B x T x * format.
        
    output, hidden = self.rnn(embedding, [1])
    #output from GRU is: output-tensor containing output features from last 
    #layer of GRU for each timestep,
    #and a tensor containing the hidden state for t = seq_len
    
    #https://pytorch.org/docs/stable/nn.html#pad-packed-sequence
    #Pads a packed batch of variable length sequences.
    #It is an inverse operation to pack_padded_sequence().
    #where T is the length of the longest sequence and B is the batch size. 
    #If batch_first is True, the data will be transposed into B x T x * format.
    #takes as input: sequence (PackedSequence) – batch to pad
    #batch_first (bool, optional) – if True, the output will be in B x T x * format.
    #returns: a Tuple of Tensor containing the padded sequence, 
    #and a Tensor containing the list of lengths of each sequence in the batch.
    output, _ = pad_packed_sequence(output)
    
    return output, hidden
  
  def initHidden(self):
    #bidirection x batch size x hidden size 
    return torch.zeros(2, batch_size, self.hidden_size)

In [0]:
my_enc = MyEncoderRNN(50, 256)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
out, hid = my_enc(embedded, my_enc.initHidden)

The decoder is simply another RNN that takes encoder output vector and outputs a sequence of words. In its simplest form it takes the context vector (last encoder output) as the initial hidden state of the decoder. At every time step, the decoder is given the embedding of a token and a hidden state (that'd be the last encoder output if its the first time step). NB: usually its a "sos" token. 

There is a problem with transduction bottlenecks when we try to encode the entire sentence with a single vector output from the encoder. The attention mechanism can overcome this by allowing us to focus on different parts of the encoder output for each decoding time step. We do so by calculating the attention weights. Then multiply these with the encoder output vectors to create a weighted sum `attn_applied`, that contains the information about that specific part of the input sequence. 

![alt text](https://i.imgur.com/1152PYf.png)

As mentioned, we do this via a feed-forward layer `attn` (decoder inputs and hidden state as inputs) but since sequences are of variable length we need a max sentence length. 

![alt text](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

The encoder reads an input sequence one item at a time, and outputs a vector at each step. The final output of the encoder is kept as the context vector. The decoder uses this context vector to produce a sequence of outputs one step at a time. The fixed-length vector carries the burden of encoding the the entire "meaning" of the input sequence, no matter how long that may be. The attention mechanism introduced by Bahdanau et al. addresses this by giving the decoder a way to "pay attention" to parts of the input, rather than relying on a single vector. For every step the decoder can select a different part of the input sentence to consider. Attention is calculated with another feedforward layer in the decoder. This layer will use the current input and hidden state to create a new vector, which is the same size as the input sequence (in practice, a fixed maximum length). This vector is processed through softmax to create attention weights, which are multiplied by the encoders' outputs to create a new context vector, which is then used to predict the next output.

Attention assigns context elements attention weights which define a weighted sum over context representations. This is known as content-based attention where attention weights are computed by comparing the current time-step to all elements in the context. Computing unlimited context size with attetnion is quadratic in complexity in the input length. 
Lightweight convolutions may overcome this which increase linearly with input size. Weights are shared across the channel dimension. This reduces the amount of weights compared to standard non-separable convolutions. Lightweight convs reuse the same weights for context elements regardless of timestep. 

Dynamic convs build on this by predicting different conv kernel at each time step. It is a function of only the current time step.

self attention of Vaswani et al. 2017 applies three projection to the input to obtain a key (K), value (V) representations, where n is the number of time steps, d the input/output dimension. H is the number of heads where each head can learn separate attention weights over dk features and attend to different positions. It compures dot-products between key/query pairs, scales to stabilize in training and softmax normalizes the result. Finally, computes a weighted sum using the output of the value projection (V). 




BERT, instead of reading a text left to right or right to left bidirectionally, it reads the entire sequence at once, allowing the model to capture the context of a word. You feed the transformer a sequence of token embeddings and it outputs a sequence of vectors of size H (each vector corresponds to an input token with the same index). It is used to train a language model whereby you try predict the next word (conditional probability) given a previous word or words (and you do the product over this). But BERT masks tokens before training and this forces bert to predict the original value of the masked word based on non-masked words. This means there is a classification layer ontop of the encoder output. Then multiplying the output vectors by the embedding matrix, transforming them into the fixed vocabulary dimensions. Then calculating the probability of each word in the vocab with softmax. BERT's loss only takes the masked word predictions and is slower to converge but adds more contextual awareness. 

In training Bert, the model has pairs of sentences as input and tries to predict if the second sentence in the pair is the subsequent sentence in the original. We split inputs 50/50 as pairs that are and are not subsequent sentences. The input is processed with a [CLS] token at the beginning of the first sentence and a [SEP] token at the end of each sentence. A sentence embedding to indicate if it is sentence A or B to each token. A positional embedding is added to each token to indicate its position in the sequence. To predict: the entire input sequence goes through the transformer, the output of the [CLS] token is transformed into a 2by1 vector, using a classification layer. Then calculating the probability of is it the next sentence with softmax. The Masked LM and Next Sentence Prediction are trained together with the goal of minimizing the combined loss function of the two. 

BERT can be used for a wide variety of language tasks and finetuned: Classification by simply adding a classfication layer on top of the transformer output for the [CLS] token. 
BERT is huge with 345 million parameters, taking 16 TPUs 4 days. 

In [0]:
class MyAttentionDecoderRNN(nn.Module):
  
  def __init__(self):
    super(MyAttentionDecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_emb_size = embed_size
    self.dropout_perc = dropout_perc
    
    self.attn = nn.Linear(self.hidden_size * 2, self.max_length)#*2 for bidir
    self.attn_comb = nn.Linear(self.hidden_size * 2, self.hidden_size)
    
    self.dropout = nn.Dropout(self.dropout_perc)
    self.gru = nn.GRU(embed_size, hidden_size, dropout=dropout)
    self.max_length = max_length
    self.out = nn.Linear()
    # provide coverage as input when computing enc attn?
    self.enc_attn_cover = enc_attn_cover
    # pointer net
    self.pointer = pointer
    
    #using glove embedding so its vocab size by embedding dim
    
  def forward(self, embedding, input, encoder_output_vector):
    # Get the embedding of the current input word (last output word)
    embedded = self.embedding(input).view(1, 1, -1)# S=1 x B x N
    embedded = self.dropout(embedded)
    
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                            encoder_outputs.unsqueeze(0))
    
    
    output, hidden = self.gru(output, hidden)
    
    

In [0]:
class DecoderRNN(nn.Module):

  def __init__(self, vocab_size, embed_size, hidden_size, *, enc_attn=True, dec_attn=True,
               enc_attn_cover=True, pointer=True, tied_embedding=None, out_embed_size=None,
               in_drop: float=0, rnn_drop: float=0, out_drop: float=0, enc_hidden_size=None):
    super(DecoderRNN, self).__init__()
    self.vocab_size = vocab_size
    self.hidden_size = hidden_size
    self.combined_size = self.hidden_size
    self.enc_attn = enc_attn
    self.dec_attn = dec_attn
    self.enc_attn_cover = enc_attn_cover
    self.pointer = pointer
    self.out_embed_size = out_embed_size
    if tied_embedding is not None and self.out_embed_size and embed_size != self.out_embed_size:
      print("Warning: Output embedding size %d is overriden by its tied embedding size %d."
            % (self.out_embed_size, embed_size))
      self.out_embed_size = embed_size

    self.in_drop = nn.Dropout(in_drop) if in_drop > 0 else None
    self.gru = nn.GRU(embed_size, self.hidden_size, dropout=rnn_drop)

    if enc_attn:
      if not enc_hidden_size: enc_hidden_size = self.hidden_size
      self.enc_bilinear = nn.Bilinear(self.hidden_size, enc_hidden_size, 1)
      self.combined_size += enc_hidden_size
      if enc_attn_cover:
        self.cover_weight = nn.Parameter(torch.rand(1))

    if dec_attn:
      self.dec_bilinear = nn.Bilinear(self.hidden_size, self.hidden_size, 1)
      self.combined_size += self.hidden_size

    self.out_drop = nn.Dropout(out_drop) if out_drop > 0 else None
    if pointer:
      self.ptr = nn.Linear(self.combined_size, 1)

    if tied_embedding is not None and embed_size != self.combined_size:
      # use pre_out layer if combined size is different from embedding size
      self.out_embed_size = embed_size

    if self.out_embed_size:  # use pre_out layer
      self.pre_out = nn.Linear(self.combined_size, self.out_embed_size)
      size_before_output = self.out_embed_size
    else:  # don't use pre_out layer
      size_before_output = self.combined_size

    self.out = nn.Linear(size_before_output, vocab_size)
    if tied_embedding is not None:
      self.out.weight = tied_embedding.weight

  def forward(self, embedded, hidden, encoder_states=None, decoder_states=None, coverage_vector=None, *,
              encoder_word_idx=None, ext_vocab_size: int=None, log_prob: bool=True):
    """
    :param embedded: (batch size, embed size)
    :param hidden: (1, batch size, decoder hidden size)
    :param encoder_states: (src seq len, batch size, hidden size), for attention mechanism
    :param decoder_states: (past dec steps, batch size, hidden size), for attention mechanism
    :param encoder_word_idx: (src seq len, batch size), for pointer network
    :param ext_vocab_size: the dynamic vocab size, determined by the max num of OOV words contained
                           in any src seq in this batch, for pointer network
    :param log_prob: return log probability instead of probability
    :return: tuple of four things:
             1. word prob or log word prob, (batch size, dynamic vocab size);
             2. RNN hidden state after this step, (1, batch size, decoder hidden size);
             3. attention weights over encoder states, (batch size, src seq len);
             4. prob of copying by pointing as opposed to generating, (batch size, 1)
    Perform single-step decoding.
    """
    batch_size = embedded.size(0)
    combined = torch.zeros(batch_size, self.combined_size, device=DEVICE)

    if self.in_drop: embedded = self.in_drop(embedded)

    output, hidden = self.gru(embedded.unsqueeze(0), hidden)  
    # unsqueeze and squeeze are necessary
    combined[:, :self.hidden_size] = output.squeeze(0)        
    # as RNN expects a 3D tensor (step=1)
    offset = self.hidden_size
    enc_attn, prob_ptr = None, None  # for visualization

    if self.enc_attn or self.pointer:
      # energy and attention: (num encoder states, batch size, 1)
      num_enc_steps = encoder_states.size(0)
      enc_total_size = encoder_states.size(2)
      enc_energy = self.enc_bilinear(hidden.expand(num_enc_steps, batch_size, -1).contiguous(),
                                     encoder_states)
      if self.enc_attn_cover and coverage_vector is not None:
        enc_energy += self.cover_weight * torch.log(coverage_vector.transpose(0, 1).unsqueeze(2) + eps)
      # transpose => (batch size, num encoder states, 1)
      enc_attn = F.softmax(enc_energy, dim=0).transpose(0, 1)
      if self.enc_attn:
        # context: (batch size, encoder hidden size, 1)
        enc_context = torch.bmm(encoder_states.permute(1, 2, 0), enc_attn)
        combined[:, offset:offset+enc_total_size] = enc_context.squeeze(2)
        offset += enc_total_size
      enc_attn = enc_attn.squeeze(2)

    if self.dec_attn:
      if decoder_states is not None and len(decoder_states) > 0:
        dec_energy = self.dec_bilinear(hidden.expand_as(decoder_states).contiguous(),
                                       decoder_states)
        dec_attn = F.softmax(dec_energy, dim=0).transpose(0, 1)
        dec_context = torch.bmm(decoder_states.permute(1, 2, 0), dec_attn)
        combined[:, offset:offset + self.hidden_size] = dec_context.squeeze(2)
      offset += self.hidden_size

    if self.out_drop: combined = self.out_drop(combined)

    # generator
    if self.out_embed_size:
      out_embed = self.pre_out(combined)
    else:
      out_embed = combined
    logits = self.out(out_embed)  # (batch size, vocab size)

    # pointer
    if self.pointer:
      output = torch.zeros(batch_size, ext_vocab_size, device=DEVICE)
      # distribute probabilities between generator and pointer
      prob_ptr = F.sigmoid(self.ptr(combined))  # (batch size, 1)
      #prob_ptr = torch.sigmoid(self.ptr(combined))
      prob_gen = 1 - prob_ptr
      # add generator probabilities to output
      gen_output = F.softmax(logits, dim=1)  # can't use log_softmax due to adding probabilities
      output[:, :self.vocab_size] = prob_gen * gen_output
      # add pointer probabilities to output
      ptr_output = enc_attn
      output.scatter_add_(1, encoder_word_idx.transpose(0, 1), prob_ptr * ptr_output)
      if log_prob: output = torch.log(output + eps)
    else:
      if log_prob: output = F.log_softmax(logits, dim=1)
      else: output = F.softmax(logits, dim=1)

    return output, hidden, enc_attn, prob_ptr